import necessary libs/functions.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random
import time

import sys
sys.path.append("..")
from data import get_dataset # custom helper function to get dataset

set the random seeds for deterministic results.

In [2]:
SEED = 2333
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

load train, val, test datasets

In [3]:
train_data, val_data, test_data = get_dataset(["train", "val", "test"])

In [4]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

205942
39472
78143


use single GPU or CPU depending on the environment

In [5]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
dev

device(type='cuda', index=0)

use dataloader to get batches of data

In [7]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=6)

In [8]:
print(len(train_loader) * BATCH_SIZE)

205952


In [9]:
train_iter = iter(train_loader)
x, y = train_iter.next()
print(x.size(), type(x), y.size(), type(y))
print(x.dtype)

torch.Size([16, 20, 2]) <class 'torch.Tensor'> torch.Size([16, 30, 2]) <class 'torch.Tensor'>
torch.float32


In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(20 * 2, 100),
            nn.ReLU(),
            nn.Linear(100, 30 * 2)
        )
    
    def forward(self, x):
        # convert (batch_size, 20, 2) to (batch_size, 20 * 2)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [11]:
model = MLP()
print(model)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=40, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=60, bias=True)
  )
)


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [ ]:
import os
model.to(dev)
epoches = 1000
print("start training...")
for epoch in range(epoches):
    start = time.time()
    model.train()
    for i, (xb, yb) in enumerate(train_loader):
        xb = xb.to(dev)
        yb = yb.to(dev).view(yb.size(0), -1)
        
        optimizer.zero_grad()
        yb_pred = model(xb)
        loss = loss_fn(yb_pred, yb)
        loss.backward()
        optimizer.step()
        
        if i % 1000 == 0:
            print("epoch {}, round {}/{} train loss: {:.4f}".format(epoch, i, len(train_loader), loss.item()))
            
    model.eval()
    model_dir = "../saved_model/MLP"
    os.makedirs(model_path, exist_ok=True)
    torch.save(model.state_dict(), model_dir + "/MLP_epoch{}".format(epoch))
    print("start validating...")
    with torch.no_grad():
        val_loss = sum(loss_fn(model(xb.to(dev)), yb.to(dev).view(yb.size(0), -1)) for xb, yb in val_loader)
    print("epoch {}, val loss: {:.4f}, time spend: {}s".format(
            epoch, val_loss / len(val_loader), time.time() - start))

start training...
epoch 0, round 0/12872 train loss: 212.9060
epoch 0, round 1000/12872 train loss: 209.8552
epoch 0, round 2000/12872 train loss: 141.7605
epoch 0, round 3000/12872 train loss: 298.7005
epoch 0, round 4000/12872 train loss: 106.5699
epoch 0, round 5000/12872 train loss: 169.1445
epoch 0, round 6000/12872 train loss: 270.7253
epoch 0, round 7000/12872 train loss: 88.2945
epoch 0, round 8000/12872 train loss: 303.0291
epoch 0, round 9000/12872 train loss: 174.8775
epoch 0, round 10000/12872 train loss: 107.5720
epoch 0, round 11000/12872 train loss: 126.4861
epoch 0, round 12000/12872 train loss: 166.0462
start validating...
epoch 0, val loss: 2119.2402, time spend: 175.2303831577301s
epoch 1, round 0/12872 train loss: 264.5938
epoch 1, round 1000/12872 train loss: 144.3651
epoch 1, round 2000/12872 train loss: 144.9422
epoch 1, round 3000/12872 train loss: 117.1763
epoch 1, round 4000/12872 train loss: 111.5341
epoch 1, round 5000/12872 train loss: 123.6577
epoch 1, rou